# Maize Selection Trade-offs

Jupyter notebook describes ecohydrological model developed by Natasha Krell, Noah Spahn and Kelly Caylor to demonstrate crop water use of maize under various rainfall climatologies in central Kenya. The notebook below runs the model, simulates rainfall, outputs the results, and generates figures.

Last updated: 5 February 2020

## 1. Background
From Ridge to Reef notebook, Chapter 3.

### Tradeoffs between days to maturity and exposure to climate variability

This tradeoff is harder to quantify. Instead, we will model the interaction between days to maturity and crop failure. Our strategy will be to examine a very field-scale water balance that looks only at relative water availability \[mm/mm\], $s$, daily rainfall, $R$ \[mm/day\], and daily water demand, which we will characterize as reference evapotranspiration, $ET_0$ \[mm/day\]. Because temperature is relatively constant, we will assume that $ET_0$ is constant. Therefore our water balance is simply:


$$ \frac{dS}{dt} = R(t) - ET_0 - L(s,t) - Q(s,t)$$

This formulation ignores many important processes related to soil water storage and plant response to drought. As defined, it is essentially a cumulative dryness index; during periods of the season when $\sum_t R(t) > t \times ET_0$, there will accumulation of water in the soil. During periods when $\sum_t R(t) < t \times ET_0$, the soil will dry out. 

Because the amount of plant available water in the soil in bounded between zero and some maximum, we add an additional parameter, $S_{max}$, which is the maximum amount of water `[mm]` that can be stored in the root zone. Now our model looks like this:

$$
\begin{eqnarray}
    \frac{dS}{dt} &=& R(t) - ET_0 - L(s,t) - Q(s,t) & if & (0 \leq S \leq S_{max}) \\
    & & & else; \\
    \frac{dS}{dt} &=& 0 
\end{eqnarray}       
$$

Assuming a typical value of porosity is 0.4 and soil depth is 500 mm (0.5 meters), the value of $S_{max}$ is 200, here is what our model looks like in python:

```python
n = 0.4  # Porosity, [m3/m3]
Zr = 500 # Rooting depth [mm]
S_max = n*Zr    # Max soil water storage [mm]
S[0] = 30  # Initial soil water storage [mm]
ET_0 = 6.5 # Daily reference evapotranspiration [mm]

S = np.zeros(len(R)) # Pre-allocate the array for S
dSdt = np.zeros(len(R))# Pre-allocate the array for dSdt

for t in range(len(R)):
    dSdt[t] = R[t] - ET_0
    S[t+1] = S[t] + dSdt[t]
    if S[t+1] < S_max:
        S[t+1] = S_max
    if S[t+1] < 0:
        S[t+1] = 0
```

## 2. Import ecohydro model

Model is saved as `runmodel.py` in folder. Evaporation and transpiration are seperated to calculate ET. 

In [1]:
# import packages and set working directory
import numpy as np
import matplotlib.pyplot as plt
import os
from math import exp
#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel

In [2]:
# initialize objects
climate = Climate() # uses default climate values
soil = Soil('loam')
crop = Crop(soil=soil) # previously kc_max=1.2, LAI_max=2.0, T_max=4.0
soil.set_nZr(crop)  
model = CropModel(crop=crop,soil=soil,climate=climate)

model.run()
model.output()

done


,kc,LAI,stress,R,s,E,ET,T,L,dsdt
0,0.300000,0.750000,1.000000,0.000000,0.300000,0.099097,0.099097,0.000000,0.0,-0.099097
1,0.300000,0.750000,1.000000,0.000000,0.299561,0.098010,0.098010,0.000000,0.0,-0.098010
2,0.300000,0.750000,1.000000,0.000000,0.299126,0.096939,0.096939,0.000000,0.0,-0.096939
3,0.300000,0.750000,1.000000,10.275780,0.298696,0.095884,0.095884,0.000000,0.0,10.179896
4,0.300000,0.750000,0.912705,1.208791,0.343840,0.225620,0.279193,0.053573,0.0,0.929598
5,0.300000,0.750000,0.887473,0.000000,0.347962,0.239188,0.308719,0.069531,0.0,-0.308719
6,0.300000,0.750000,0.895813,0.000000,0.346593,0.234653,0.298884,0.064231,0.0,-0.298884
7,0.300000,0.750000,0.903925,0.000000,0.345268,0.230290,0.289390,0.059100,0.0,-0.289390
8,0.300000,0.750000,0.911814,0.000000,0.343984,0.226091,0.280224,0.054133,0.0,-0.280224
9,0.300000,0.750000,0.919485,0.000000,0.342742,0.222051,0.271373,0.049322,0.0,-0.271373


### a. Inspecting the model

In [3]:
# this should be the first term of LAI
pow(3.0/1.2,1)*0.3

# this should be the first E term
6.5*exp(-6.5*0.75)

0.04962811242258975

### b. Model tests

Something is wrong with `s`, `theta`, or `psi`, which are all methods of Soil. To figure out which of these might be causing the issue, we can test various relative soil moisture values using this figure from Liao et al. 2001b showing the relationship between relative soil moisture (x-axis: low levels of soil moisture = 0 and max soil moisture = 1) and evapotranspiration (y-axis: the sum of losses from both plant transpiration and soil evaporation).

![Create figure 5 from Liao et al. 2001b](output/Liao2001bFig5.jpg)

 When soil moisture is between field capacity (s_fc) and 1, the soil moisture is sufficient and maximum ET, (E_max), is independent of soil moisture, (s). Between field capacity (s_fc) and S* (s_star), both plant transpiration and soil evaporation result in soil moisture losses. S* is the point at which soil water loss is due to both transpiration and evaporation before soil moisture levels decrease enough for transpiration to be reduced. When soil moisture is below S*, plants start reducing transpiration: they close their stomata to prevent water loss, and thus there is a linear decrease to wilting point (s_w) as photosynthesis-related transpiration and water uptake from roots slows until reaching wilting point. At wilting point there are very low levels of soil moisture, and there is only evaporation. Between wilting point and the hygroscopic point (s_h), soil water is depleteed only by evaporation.

We wrote tests (see test_sand.py) based on the following questions:

**1. Is the hygroscopic soil moisture (s_h) less than soil moisture at wilting point (s_w)?**

```python

print(soil.sh)
print(crop.sw)

if soil.sh <= crop.sw:
    print("True")
```

**2. Is the conversion from mm to MPa is correct for sw? TODO**
**3. Is the conversion for s_star correct? TODO**

```python

print(crop.sw)
print(crop.sw_MPa)

print(crop.s_star)
print(crop.s_star_MPa)

```

**4. Is field capacity less than 1?**

```python
if soil.sfc <= 1:
    print("True")

pow((soil.sfc - soil.sh)/(1-soil.sh),1)*100
```

**5. Do the hygroscopic values correspond with expected values from Liao et al. 2001b, Figure 6: ca. 0.07 for loamy sand, and 0.19 for loam?**

```python
# Loamy sand
# initialize objects
climate = Climate() # uses default climate values
soil = Soil('loamy sand')
crop = Crop(soil=soil) 
soil.set_nZr(crop)
model = CropModel(crop=crop,soil=soil,climate=climate)
print(soil.sh)

# Loam
climate = Climate() # uses default climate values
soil = Soil('loam')

crop = Crop(soil=soil) 
soil.set_nZr(crop)
model = CropModel(crop=crop,soil=soil,climate=climate)

print(soil.sh)

```

In [4]:
pow(0.05/(0.0417*60*24),1/(2*5.39+3))

0.5977531541625819

### c. Other space to mess around

In [ ]:
# Check that the value of E_w (evaporation at wilting point) is less than E_max (max evapotranspiration)

# where to get E_w?
#climate.calc_E(soil.sh)

soil.s

soil.s(0.3)

soil.s

# this is a method that returns the relative soil moisture value given the volumetric water content.
# the input is theta which is an interval between 0 and porosity = 0.395

soil.n

# let's try and get linspace 0 to soil.n

PRECISION = 2

theta_values = np.linspace(0.1, soil.n, num=100).round(3)
theta_values
#len(theta_values)

#TODO: how to fix this: if I start with zero it throws an error below?

vals = np.array([soil.s(theta_) for theta_ in theta_values])

d = np.arange(101)
plt.plot(vals, '-')
plt.title('Hmmm')
plt.xlabel('Hmm')
plt.show()

In [ ]:
soil = Soil('sand')
soil.set_nZr(crop)

print(crop.s_star)
print(soil.sfc)

soil = Soil('clay')
soil.set_nZr(crop)

print(crop.s_star)
print(soil.sfc)


soil = Soil('loam')
soil.set_nZr(crop)

print(crop.s_star)
print(soil.sfc)

In [ ]:
soil.calc_L(0.395)

self = soil
self.Psi_L_MPa
self.theta(-1.5)
#self.n
#self.sfc
#self.psi(0.44*0.37) * 1E6 / 1000 / 9.8
pow(0.05/60/24/self.Ks,1/(2*self.b+3))*self.n

In [ ]:
self = soil

In [ ]:
# run for three maize varieties (early, med, late harvesting) and multiple seasons.
kc_collection = {}

for season_length in range(90, 120, 180):
  climate = Climate(t_seas=season_length)
  soil = Soil('sand')
  crop = Crop(soil=soil)
  soil.set_nZr(crop)
  model  = CropModel(crop=crop,soil=soil,climate=climate)
  model.run()
  kc_key = str(season_length)+"kc"
  kc_collection[kc_key] = model.kc
    
# now want to repeat this process but for multiple rainfall climatologies, soil types, different output variables, etc. etc.

In [ ]:
kc_collection

In [ ]:
model.n_days

In [ ]:
climate = Climate()
soil

In [ ]:
climate.calc_E(0.3, 1, soil=soil, plant=crop)

from math import exp
climate.ET_max*exp(-0.5*crop.calc_LAI(1)/crop.LAI_max)

(0.3-soil.sh)/(1-soil.sh)
soil.theta(-1000)

crop.s_star

crop.s_star_MPa


In [ ]:
soil.Psi_S_MPa

In [ ]:
soil = Soil('loamy sand')
soil.theta(-10)/soil.n


In [ ]:
# make some figures
print(model.E)
model.ET
model.ET_max
model.s

len(model.ET_max)

In [ ]:
model.T_max

In [ ]:
# other

model_output = ['R', 'ET']
A = pd.DataFrame(np.zeros((120,len(model_output))), columns=model_output)
B = pd.DataFrame(np.zeros((120,len(model_output))), columns=model_output)
C = A.append(B)
